In [126]:
import pandas as pd 
import numpy as np 
from datetime import date

# Graficos 
import matplotlib.pyplot as plt 
#import plotly.graph_objects as go
#from plotly.subplots import make_subplots
import plotly.express as px

# Funciones

In [66]:
def pareto(df,col,Title):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    #construir una tabla de frecuencias. 
    
    rating_counts = (df.groupby(col).agg(frec_absoluta=(col, "count"))).sort_values(['frec_absoluta'], ascending = False)
    
    rating_counts["frec_absoluta_acumulada"] = rating_counts["frec_absoluta"].cumsum()

    
    rating_counts['frec_relativa'] = rating_counts['frec_absoluta'].apply(lambda x: x/rating_counts['frec_absoluta'].sum())
    rating_counts['frec_relativa(%)'] = rating_counts['frec_relativa'].apply(lambda x: x*100 )

    
    
    rating_counts['frec_relativa_acumulada'] = rating_counts['frec_absoluta_acumulada'].apply(lambda x: x/rating_counts['frec_absoluta'].sum())  
    rating_counts['frec_relativa_acumulada(%)'] = rating_counts['frec_relativa_acumulada'].apply(lambda x: x*100)
    
    #diagrama pareto
    trace1 = go.Bar(
        x=[str(x) for x in rating_counts.index],
        y=rating_counts['frec_relativa(%)'],
        name='frec_relativa',
        marker=dict(
            color='rgb(34,163,192)'
                   )
    )
    trace2 = go.Scatter(
        x=[str(x) for x in rating_counts.index],
        y=rating_counts['frec_relativa_acumulada(%)'],
        name='fre_relativa_acumulada',
        yaxis='y2'

    )

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(trace1)
    fig.add_trace(trace2,secondary_y=True)
    fig['layout'].update(height = 600, width = 800, title =Title,xaxis=dict(
          tickangle=-90
        ))


    fig.show()

In [87]:
def conv(x):
    meses = {'enero':'01','febrero':'02','marzo':'03','abril':'04','mayo':'05','junio':'06','julio':'07','agosto':'08',
            'septiembre':'09','octubre':'10','noviebre':'11','diciembre':'12'}
    if x == None:
        return 0
    for mes in meses:
        if mes == x:
            return meses[x]

In [ ]:
def con_categ_to_columns(df,col,col_fecha):
    data = pd.DataFrame({})
    uniques = df[col].unique()
    df = df[[col_fecha,col]]
    for unique in uniques:
        aux = df[df[col] == unique]
        aux2 = (aux.groupby(col_fecha).agg(count=(col, "count"))).sort_values([col_fecha], ascending = True)
        data[unique] = aux2['count'] 
    data['fecha'] = aux2.index
    return data

# Primer Acercamiento

In [157]:
df = pd.read_csv('https://raw.githubusercontent.com/RiemanNClav/Hechos-de-Transito-registrados-por-la-SSC/master/hechos_transito.csv',)

C:\Users\XMX5634\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (4,16,17,18,19,20,32,36,41,42,43,44,45,47,49,50,51,52,53,54,66,67,68) have mixed types.Specify dtype option on import or set low_memory=False.



In [158]:
df.head(1)

,no_folio,fecha_evento,mes,hora,condicion,tipo_de_evento,coordenada_x,coordenada_y,punto_1,punto_2,...,ciclista_lesionado,pasajero_occiso,pasajero_lesionado,motociclista_occiso,motociclista_lesionado,conductor_occiso,conductor_lesionado,diagnostico_occiso,diagnostico_lesionado,origen
0,BJ/200406/03499,2020-04-06,abril,12:50:00,NaN,CHOQUE,19.368116,-99.142903,EJE 7 SUR,ANTILLAS,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,POLICONTUNDIDO,RADIO


In [159]:
df.shape

(107882, 69)

In [160]:
df.isnull().sum()/len(df)*100

no_folio                  0.000927
fecha_evento              0.000000
mes                       0.000000
hora                      0.027808
condicion                67.517287
                           ...    
conductor_occiso         32.482713
conductor_lesionado      32.482713
diagnostico_occiso       98.764391
diagnostico_lesionado    33.404090
origen                   32.482713
Length: 69, dtype: float64

# Limpieza de Datos

In [161]:
df['year'] = pd.to_datetime(df['fecha_evento']).dt.to_period('Y')

In [162]:
df['num_mes'] = df['mes'].apply(lambda x: conv(x))

In [163]:
fechas = df['year'].astype(str) + "-" + df['num_mes'] + "-" + '01'
df['fecha_mensual'] = pd.to_datetime(fechas)

# Exploración de Datos 

In [257]:
pareto(df,'tipo_de_evento','Eventos ocurridos en CDMX')

Se observa que el 90% de los accidentes de tránsito que ocurren en la CDMX son por causas como CHOQUES, ATROPELLOS Y DERRAPOS, en donde el 56% son provocados por CHOQUES. 

Valdria la pena revisar estas tres categorías por separado y las ultimas 3 en conjunto (CAIDA DE CICLISTA, VOLCADURA Y CAIDA DE PASAJERO) ya que aproximadamente el 5% del total, son accidentes de este estilo. 

In [245]:
u = con_categ_to_columns(df,'tipo_de_evento','fecha_mensual')

In [256]:
fig = px.line(u, x="fecha", y=u.columns,
              hover_data={"fecha": "|%B %d, %Y"},
              title='Serie de Tiempo de cada uno de los eventos ocurridos en CDMX',
             width=1300, height=400)
fig.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y")
fig.show()

In [9]:
df2 = df[(df['tipo_de_evento'] == 'CHOQUE') | (df['tipo_de_evento'] == 'ATROPELLADO') | (df['tipo_de_evento'] == 'DERRAPADO')]

In [10]:
df2.loc[df2['condicion'].isnull(), 'condicion'] = 'DESC'

C:\Users\XMX5634\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
df2['evento'] = df2['condicion'].apply(lambda x: x[0]) +'_'+ df2['tipo_de_evento']

C:\Users\XMX5634\AppData\Local\Temp/ipykernel_23812/2525404346.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
pareto(df2[df2['condicion'] != 'DESC'],'evento','diagrama pareto de los eventos ocurridos')

In [172]:
df3 = df2[df2['tipo_de_evento'] == 'ATROPELLADO']
df3 = df3.rename(columns = {'tipo_de_evento': 'ATROPELLADO'})